In [ ]:
import mesmerize_core as mc
import mesmerize_viz
import fastplotlib as fpl

import numpy as np
from copy import deepcopy
import pandas as pd

**You will need `mesmerize-viz` installed for the visualizations**

In [ ]:
pd.options.display.max_colwidth = 120

# Paths

`mesmerize-core` helps manage the outputs of caiman algorithms and organizes "parameter variants" - the output of a given combination of input data and algorithm parameters. In order to run the algorithms you must tell `mesmerize-core` where your _input data_ are located and decide on a **top level raw data directory**. For example consider the following directory structure of experimental data (you may organize your raw data however you wish, this is just an example). We can see that all the experimental data lies under `/data/group_name/my_name/exp_data`. Therefore we can use this `exp_data` dir as a `parent raw data path`. `mesmerize-core` will then only store the _relative_ paths to the raw data files, this allows you to move datasets between computers and filesystems. `mesmerize-core` does not store any hard file paths, only relative paths.

```
/data/group_name/my_name
                        └── exp_data
                            ├── axonal_imaging
                            │   ├── mouse_1
                            │   │   ├── exp_a.tiff
                            │   │   ├── exp_b.tiff
                            │   │   └── exp_c.tiff
                            │   ├── mouse_2
                            │   │   ├── exp_a.tiff
                            │   │   └── exp_b.tiff
                            │   └── mouse_3
                            └── hippocampus_imaging
                                ├── mouse_1
                                │   ├── exp_a.tiff
                                │   ├── exp_b.tiff
                                │   └── exp_c.tiff
                                ├── mouse_2
                                └── mouse_3
```

**For this demo set the `caiman_data` dir as the parent raw data path**

Sidenote: We recommend using [pathlib](https://docs.python.org/3/library/pathlib.html) instead of manually managing paths as strings. `pathlib` is just a part of the Python standard library, it makes it much easier to deal with paths and saves a lot of time in the long-run! It also makes your paths compatible across operating systems. Therefore even if you are on Windows you can use the regular `/` for paths, you do not have to worry about the strangeness of `\\` and `\`

In [ ]:
mc.set_parent_raw_data_path("/home/kushal/caiman_data/")

### Batch path, this is where caiman outputs will be organized

This can be anywhere, it does not need to be under the parent raw data path.

In [ ]:
batch_path = mc.get_parent_raw_data_path().joinpath("mesmerize-batch/batch.pickle")

# Create a new batch

This creates a new pandas `DataFrame` with the columns that are necessary for mesmerize. In mesmerize we call this the **batch DataFrame**. You can add additional columns relevant to your experiment, but do not modify columns used by mesmerize.

Note that when you create a DataFrame you will need to use `load_batch()` to load it later. You cannot use `create_batch()` to overwrite an existing batch DataFrame

In [ ]:
# create a new batch
df = mc.create_batch(batch_path)

# to load existing batches use `load_batch()`
# df = load_batch(batch_path)

# View the dataframe

It is empty and has the required columns for mesmerize

In [ ]:
df

# Let's add stuff to the DataFrame!

First get an input movie. An input movie must be somewhere under `parent raw data path`. It does not have to be directly under `parent raw data path`, it can be deeply nested anywhere under it.

In [ ]:
# We'll use the Sue movie from caiman
# download it if you don't have it
from caiman.utils.utils import download_demo
download_demo()

In [ ]:
movie_path = mc.get_parent_raw_data_path().joinpath("example_movies/Sue_2x_3000_40_-46.tif")

# Motion correction parameters

The parameters are passed **directly** to `caiman`, this means you need to use the same exact names for the parameters and you can use all the parameters that you can use with `caiman` - because it's just passing them to `caiman`.


The parameters dict for a mesmerize batch item must have the following structure. Put all the parameters in a dict under a key called **main**. The **main** dict is then fed directly to `caiman`.

```python
{"main": {... params directly passed to caiman}}
```

In [ ]:
# We will start with one version of parameters
mcorr_params1 =\
{
  'main': # this key is necessary for specifying that these are the "main" params for the algorithm
    {
        'max_shifts': [24, 24],
        'strides': [48, 48],
        'overlaps': [24, 24],
        'max_deviation_rigid': 3,
        'border_nan': 'copy',
        'pw_rigid': True,
        'gSig_filt': None
    },
}

# Add a "batch item" to the DataFrame this is the combination of:
* algorithm to run, `algo`
* input movie to run the algorithm on, `input_movie_path`
* parameters for the specified algorithm, `params`
* a name for you to keep track of things, usually the same as the movie filename, `item_name`

In [ ]:
# add an item to the DataFrame
df.caiman.add_item(
    algo='mcorr',
    input_movie_path=movie_path,
    params=mcorr_params1,
    item_name=movie_path.stem,  # filename of the movie, but can be anything
)

df

We can now see that there is one item in the DataFrame. What we called a "item" in `mesmerize-core` DataFrames is technically called a pandas `Series` or row.

# Run an item

There is only one item in this DataFrame and it is located at index `0`. You can run a row using `df.iloc[index].caiman.run()`

Technical notes: On Linux & Mac it will run in subprocess but on Windows it will run in the local kernel. If using the subprocess backend (only Linux & Mac) you can use `run(wait=False)` if you don't want to block the kernel while running. 

In [ ]:
df.iloc[0].caiman.run()

# Reload DataFrame from disk

After running one or any number of items in `mesmerize-core` you must call `df = df.caiman.reload_from_disk()`. This loads the DataFrame with references to the output files in the batch directory.

In [ ]:
df = df.caiman.reload_from_disk()

In [ ]:
df

# Outputs

We can see that the outputs column has been populated. The entries in this column do not have to be accessed directly. The `mesmerize-core` API allows you to fetch these outputs.

In [ ]:
index = 0 # we will fetch stuff from index 0 which we just ran

# get the motion corrected movie memmap
mcorr_movie = df.iloc[0].mcorr.get_output()
mcorr_movie.shape

In [ ]:
# path to the mcorr memmap if you ever need it
mcorr_memmap_path = df.iloc[0].mcorr.get_output_path()
mcorr_memmap_path

In [ ]:
# mean projection, max and std projections are also available
mean_proj = df.iloc[0].caiman.get_projection("mean")
mean_proj.shape

In [ ]:
# the input movie, note that we use `.caiman` here instead of `.mcorr`
input_movie = df.iloc[0].caiman.get_input_movie()
input_movie.shape

## Note on input movies

`get_input_movie()` will automatically work with most tiff files and memmaps. If you want to load other file types, you will need to pass it a function (see examples below) that returns a lazy-loadable array, or a numpy array if you have enough RAM. 


### tiff files

`get_input_movie()` wil try to use `tifffile.memmap` to lazy-load tiff files. This works for some tiff files. If `tifffile.memmap` fails, `mesmerize-core` has its own `LazyTiff` implementation that it will try to fallback on. However, some not every tiff file can be lazy-loaded so it is not gaurenteed that `LazyTiff` will work. The implementation of `LazyTiff` is quite simple and you might be able to subclass it for your specific type of tiff file: https://github.com/nel-lab/mesmerize-core/blob/master/mesmerize_core/arrays/_tiff.py 

### hdf5 files

```python
import h5py

def hdf5_reader(path):
    f = h5py.File(path)
    return f["your-key"]

input_movie = df.iloc[index].caiman.get_input_movie(hdf5_reader)
```

### avi files
    
```python
from mesmerize_core.arrays import LazyVideo

def video_reader(path):
    a = LazyVideo(path)  # you can use the other args if you want
    return a

input_movie = df.iloc[index].caiman.get_input_movie(video_reader)
```

# Visualize with mesmerize-viz!

- Random-access frames
- Slider for frame averaging over a window - useful for quality control
- adjust cmap, vmin vmax, etc.

In [ ]:
viz = df.mcorr.viz()
viz.show()

In [ ]:
viz.close()

# Visualizations are customizable

Hint: use `Shift` + `Tab` to bring up the docstring for `mcorr.viz()`.

In [ ]:
viz = df.mcorr.viz(data_options=["input", "mcorr"])
viz.show()

# Customization

We call `viz` a "Viz Container". The `McorrVizContainer` has a fastplotlib `ImageWidget` that you can access.

In [ ]:
viz.image_widget

Change colormaps

In [ ]:
viz.image_widget.cmap = "gray"

In [ ]:
viz.image_widget.cmap = "viridis"

Access subplots, graphics, etc. The full suite of the `fastplotlib` API

In [ ]:
viz.image_widget.gridplot["mcorr"]

In [ ]:
viz.image_widget.gridplot["mcorr"]["image_widget_managed"].cmap.vmax

# Optional: close the visualization to free up the GPU

Not necessary if you're on a workstation or similar with decent GPUs

In [ ]:
viz.close()

# Parameter variants - this is the purpose of mesmerize-core!

Let's add another row to the DataFrame. We will use the same input movie but different parameters. This is the basis of how we can perform a _parameter gridsearch_.

In [ ]:
mcorr_params2 =\
{
  'main':
    {
        'max_shifts': [4, 4],
        'strides': [48, 48],
        'overlaps': [24, 24],
        'max_deviation_rigid': 3,
        'border_nan': 'copy',
        'pw_rigid': True,
        'gSig_filt': None
    },
}

# add other param variant to the batch
df.caiman.add_item(
  algo='mcorr',
  item_name=movie_path.stem,
  input_movie_path=movie_path,
  params=mcorr_params2
)

df

We can see that there are two batch items for the same input movie.

# Parameter Gridsearch

Use a `for` loop to add multiple different parameter variants more efficiently.

In [ ]:
# copy the mcorr_params2 dict to make some changes
new_params = deepcopy(mcorr_params2)

# some variants of max_shifts
for shifts in [2, 32]: 
    for strides in [12, 24, 64]:
        overlaps = int(strides / 2)
        # deep copy is the safest way to copy dicts
        new_params = deepcopy(new_params)

        # assign the "max_shifts"
        new_params["main"]["max_shifts"] = (shifts, shifts)
        new_params["main"]["strides"] = (strides, strides)
        new_params["main"]["overlaps"] = (overlaps, overlaps)

        df.caiman.add_item(
          algo='mcorr',
          item_name=movie_path.stem,
          input_movie_path=movie_path,
          params=new_params
        )

In [ ]:
df

# Distinguishing parameter variants

We can see that there are many parameter variants, but it is not easy to see the differences in parameters between the rows that have the same `item_name`.

We can use the `caiman.get_params_diffs()` to see the unique parameters between rows with the same `item_name`

In [ ]:
diffs = df.caiman.get_params_diffs(algo="mcorr", item_name=df.iloc[0]["item_name"])
diffs

# Run multiple batch items.

`df.iterrows()` iterates through rows and returns the numerical index and row for each iteration

In [ ]:
for i, row in df.iterrows():
    if row["outputs"] is not None: # item has already been run
        continue # skip
        
    process = row.caiman.run()
    
    # on Windows you MUST reload the batch dataframe after every iteration because it uses the `local` backend.
    # this is unnecessary on Linux & Mac
    # "DummyProcess" is used for local backend so this is automatic
    if process.__class__.__name__ == "DummyProcess":
        df = df.caiman.reload_from_disk()

# Outputs

Load the output information into the DataFrame

In [ ]:
df = df.caiman.reload_from_disk()

In [ ]:
df

# Visualization using `mesmerize-viz` 

In [ ]:
from mesmerize_viz import *

In [ ]:
mcorr_viz = df.mcorr.viz(data_options=["input", "mcorr"])
mcorr_viz.show()

# Build your own visualizations using `fastplotlib`

# Use `ImageWidget` to view 5 mcorr results simultaneously

This type of visualization usually requires your files to be lazy-loadble, and the performance will depend on your hard drive's capabilities.

In [ ]:
# first item is just the raw movie
movies = [df.iloc[0].caiman.get_input_movie()]

# subplot titles
subplot_names = ["raw"]

# we will use the mean images later
means = [df.iloc[0].caiman.get_projection("mean")]

# add all the mcorr outputs to the list
for i, row in df.iterrows():
    # add to the list of movies to plot
    movies.append(row.mcorr.get_output())
    
    # subplot title to show dataframe index
    subplot_names.append(f"ix: {i}")
    
    # mean images which we'll use later
    means.append(row.caiman.get_projection("mean"))

# create the widget
mcorr_iw_multiple = fpl.ImageWidget(
    data=movies,  # list of movies
    window_funcs={"t": (np.mean, 17)}, # window functions as a kwarg, this is what the slider was used for in the ready-made viz
    grid_plot_kwargs={"size": (900, 550)},
    names=subplot_names,  # subplot names used for titles
    cmap="gnuplot2"
)

mcorr_iw_multiple.show()

In [ ]:
df.caiman.get_params_diffs(algo="mcorr", item_name=df.iloc[0]["item_name"])

### Modify the `window_funcs` at any time

In [ ]:
mcorr_iw_multiple.window_funcs["t"].window_size = 5

## There is some motion on the left side of of `ix: 2` at timepoint `1452`, `2037` and a few others. This will be more obvious if we substract a mean image from each frame. You can use `frame_apply` to apply a function before displaying frames in the `ImageWidget`

This can be combined with `window_funcs` or used by itself. If used in combination with `window_funcs`, the window functions are computed first and then fed to `frame_apply`.

For this example the `frame_apply` functions subtract the mean image for each movie.

General form:

```python
{
    data_ix: function() # returns 2D frame
    ...
}
```

In [ ]:
subtract_means = {
    0: lambda x: x - means[0],
    1: lambda x: x - means[1],
    2: lambda x: x - means[2],
    3: lambda x: x - means[3],
    4: lambda x: x - means[4],
    5: lambda x: x - means[5]
}

In [ ]:
mcorr_iw_multiple.frame_apply = subtract_means

### Different colormaps can make the motion more obvious

In [ ]:
for sp in mcorr_iw_multiple.gridplot:
    sp.graphics[0].cmap = "jet"

In [ ]:
# disable frame apply
mcorr_iw_multiple.frame_apply = dict()

In [ ]:
mcorr_iw_multiple.close()

# ix `3` seems to work the best so we will cleanup the DataFrame and remove all other items.

### You can remove batch items (i.e. rows) using `df.caiman.remove_item(<item_uuid>)`

**Note that this also cleans up the output data in the batch directory!**

In [ ]:
# make a list of rows we want to keep using the uuids
rows_keep = [df.iloc[3].uuid]
rows_keep

### On windows calling `remove_item()` will raise a `PermissionError` if you have the memmap file open.
### Unfortunately the current workaround is to kill the kernel if you want to delete batch items with open memmaps.

There is currently no way to close a `numpy.memmap`, even if you remove all references to it.

In [ ]:
for i, row in df.iterrows():
    if row.uuid not in rows_keep:
        df.caiman.remove_item(row.uuid)

df

### As you can see above, the numerical index changed for what was previously item 3. Indices are always reset when you use `caiman.remove_item()`. However, UUIDs are always maintained.

# CNMF

## Continue from mcorr above and perform CNMF using the good mcorr output.

First, the params for CNMF. Put the CNMF params within the `main` key, `refit` is if you want to run CNMF for a second iteration.

In [ ]:
# some params for CNMF
params_cnmf =\
{
    'main': # indicates that these are the "main" params for the CNMF algo
        {
            'fr': 30, # framerate, very important!
            'p': 1,
            'nb': 2,
            'merge_thr': 0.85,
            'rf': 15,
            'stride': 6, # "stride" for cnmf, "strides" for mcorr
            'K': 4,
            'gSig': [4, 4],
            'ssub': 1,
            'tsub': 1,
            'method_init': 'greedy_roi',
            'min_SNR': 2.0,
            'rval_thr': 0.7,
            'use_cnn': True,
            'min_cnn_thr': 0.8,
            'cnn_lowest': 0.1,
            'decay_time': 0.4,
        },
    'refit': True, # If `True`, run a second iteration of CNMF
}

### Add a single cnmf item to the batch

In [ ]:
# add a batch item
df.caiman.add_item(
    algo='cnmf', # algo is cnmf
    input_movie_path=df.iloc[0],  # use mcorr output from a completed batch item
    params=params_cnmf,
    item_name=df.iloc[0]["item_name"], # use the same item name
)

### Just like with motion correction, we can use loops to add multiple parameter variants. This is useful to perform a parameter search to find the params that work best for your dataset. Here I will use `itertools.product` which is better than deeply nested loops.

In [ ]:
from itertools import product

# variants of several parameters
gSig_variants = [6, 8]
K_variants = [4, 8]
merge_thr_variants = [0.8, 0.95]

# always use deepcopy like before
new_params_cnmf = deepcopy(params_cnmf)

# create a parameter grid
parameter_grid = product(gSig_variants, K_variants, merge_thr_variants)

# a single for loop to go through all the various parameter combinations
for gSig, K, merge_thr in parameter_grid:
    # deep copy params dict just like before
    new_params_cnmf = deepcopy(new_params_cnmf)
    
    new_params_cnmf["main"]["gSig"] = [gSig, gSig]
    new_params_cnmf["main"]["K"] = K
    new_params_cnmf["main"]["merge_thr"] = merge_thr
    
    # add param combination variant to batch
    df.caiman.add_item(
        algo="cnmf",
        item_name=df.iloc[0]["item_name"],
        input_movie_path=df.iloc[0],
        params=new_params_cnmf
    )

### See that there are a lot of new cnmf batch items

In [ ]:
df

## Since it is difficult to see the different parameter variants above, we can just view the diffs

### The index numbers on the diffs correspond to the indices in the parent DataFrame above

In [ ]:
df.caiman.get_params_diffs(algo="cnmf", item_name=df.iloc[1]["item_name"])

# Run the added `cnmf` batch items

### First, this is how you can filter a pandas DataFrame using multiple columns. This gives you the rows (batch items) using the "cnmf" `"algo"` and those that match a particular `"item_name"`

In [ ]:
df[
    (df["algo"] == "cnmf") &  # algo
    (df["item_name"] == df.iloc[0]["item_name"])  # item name
]

## Run only these items

In [ ]:
for i, row in df[
    (df["algo"] == "cnmf") &
    (df["item_name"] == df.iloc[0]["item_name"])
].iterrows():
    
    process = row.caiman.run()
    
    # on Windows you MUST reload the batch dataframe after every iteration because it uses the `local` backend.
    # this is unnecessary on Linux & Mac
    # "DummyProcess" is used for local backend so this is automatic
    if process.__class__.__name__ == "DummyProcess":
        df = load_batch(df.paths.get_batch_path())

### We now have CNMF outputs

In [ ]:
df = df.caiman.reload_from_disk()
df[df["algo"] == "cnmf"]

In [ ]:
# see which batch items completed succcessfully
df[df["algo"] == "cnmf"]["outputs"].apply(lambda x: x["success"])